In [ ]:
#import libraries
#divide data into training and test set
#initialize repeated k-fold stratified on training data
#find accuracy on test set
#save the model

In [ ]:
from tune_sklearn import TuneSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from ray.tune.schedulers import MedianStoppingRule
from sklearn.model_selection import train_test_split
pipe = Pipeline([("classify",LogisticRegression(max_iter=1000,solver = 'saga',n_jobs = -1))])
param_grid = [
    {
        "classify__C": [100, 10, 1.0, 0.1, 0.01]
    },
]
scheduler = MedianStoppingRule(grace_period=10.0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)
random = TuneSearchCV(
    pipe,
    param_grid,
    search_optimization="random",
    early_stopping=scheduler,
    max_iters=50,
    pipeline_auto_early_stop=True)
random.fit(x_train,y_train)
print(random.cv_results_)

In [ ]:
y_pred = random.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
clf = LogisticRegression(C=1.0,solver='saga',max_iter = 10000,n_jobs=-1)
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_pred,y_test))

0.7670666666666667


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import json
def prepare_data(path):
  df = pd.read_csv(path)
  mat = np.matrix([x for x in  df['ELEC_embed'].apply(lambda x: json.loads(x))])
  x= np.squeeze(np.asarray(mat))
  y = df.sentiment.values
  print("Data Prepared")
  return x,y

In [ ]:
x,y = prepare_data('/content/drive/MyDrive/sentiment data/train_sent_5folds.csv')

Data Prepared


In [ ]:
y.shape

(750000,)

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 4.1MB/s 


In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
from skopt import BayesSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold,train_test_split
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.75, test_size=.25, random_state=0,stratify = y)
rskf = RepeatedStratifiedKFold(n_splits=10,n_repeats=2,random_state=1)
opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'gamma': (1e-6, 1e+1, 'log-uniform'),
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=32,
    cv=rskf
)

opt.fit(X_train, y_train)

print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

In [ ]:
Y_tr.shape

(270000,)

In [ ]:
rskf = RepeatedStratifiedKFold(n_splits=10,n_repeats=2,random_state=1)
for tr_in,ts_in in rskf.split(x,y):
  X_tr_1,X_ts_1 = x[tr_in],x[ts_in]
  Y_tr_1,Y_ts_1 = y[tr_in],y[ts_in]

In [ ]:
len(Y_ts == Y_ts_1)

30000

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold,train_test_split,cross_val_score
rskf = RepeatedStratifiedKFold(n_splits=10,n_repeats=2,random_state=1)
x_tr,x_ts,y_tr,y_ts = train_test_split(x, y, train_size=0.75, test_size=.25, random_state=0,stratify = y)

In [ ]:
#try svc()
# from sklearn.linear_model import RidgeClassifier
# from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import numpy as np
# import pandas as pd

# x,y = prepare_data('/content/drive/MyDrive/sarcasm data/train_sarc_5folds.csv')

# model = AdaBoostClassifier(n_estimators=10, random_state=1, algorithm='SAMME')
model = RandomForestClassifier()
# model = LogisticRegression(C=0.1)
# n_estimators = number of decision trees
# rf = 
# rf.fit(x_train, y_train)
# model = RandomForestClassifier(n_estimators=30, max_depth=9)
scores = cross_val_score(model, x_tr, y_tr,scoring='accuracy', cv=rskf,n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

Accuracy: 0.628 (0.002)


In [ ]:
from sklearn.metrics import accuracy_score
model.fit(x_tr,y_tr)
y_pred = model.predict(x_ts)
print(accuracy_score(y_pred,y_ts))

0.4804853333333333


In [ ]:
model.fit(x,y)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [ ]:
import joblib
filename = '/content/drive/MyDrive/sentiment data/final_rid_class.sav'
joblib.dump(model, filename)

['/content/drive/MyDrive/sentiment data/final_rid_class.sav']